In [36]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
import os
import getpass

In [37]:
load_dotenv()

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Google Gemini uchun API kalitni kiriting")

# Init a model
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
print("Model bilan aloqa bo'glandi")

Model bilan aloqa bo'glandi


In [40]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="I want to opan a resturant for uzbek food. Suggest a fancy name for this. Only one name")]).content

'Samarkand Silk'

# LangChain PromptTemplate

In [41]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

prompt_template_name.format(cuisine="Uzbek")

'I want to opan a resturant for Uzbek food. Suggest a fancy name for this. Only one name'

# Chains

### LLMChain

In [42]:
from langchain.chains import LLMChain

chain = LLMChain(llm=model, prompt=prompt_template_name)
chain.run("American")

'The Federalist Table'

In [43]:
# We might be see some warnings. Now we can try new way. LLMChain is not needed

chain = prompt_template_name | model
chain.invoke("Uzbek").content

'Lapis & Spice'

### SimpleSequentialChain

In [44]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template = "I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

new_chain = LLMChain(llm=model, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template = "Suggest some menu items for {resturant_name}. Return it as a comma separated list.")

food_item_chain = LLMChain(llm=model, prompt=prompt_template_items)


In [45]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[new_chain, food_item_chain])
response = chain.run("Uzbek")
print(response)

Altyn Meze Tabağı (Golden Mezze Platter), Acılı Ezme, Haydari, Patlıcan Salatası (Smoked Eggplant Salad), Sigara Böreği, İçli Köfte, Mercimek Çorbası (Lentil Soup), Ezogelin Çorbası, Çoban Salata (Shepherd's Salad), Adana Kebab, Urfa Kebab, Tavuk Şiş (Chicken Shish), Kuzu Şiş (Lamb Shish), Altyn Karışık Izgara (Golden Mixed Grill), Beyti Sarma, İskender Kebab, Hünkar Beğendi (Sultan's Delight), Kuzu Güveç (Lamb Casserole), Testi Kebabı (Pottery Kebab), Mantı (Turkish Dumplings), Kıymalı Pide (Minced Meat Pide), Kuşbaşılı Pide (Diced Meat Pide), Peynirli Pide (Cheese Pide), Lahmacun, Fıstıklı Baklava (Pistachio Baklava), Künefe, Sütlaç (Rice Pudding), Kazandibi, Ayran, Ev Yapımı Limonata (Homemade Lemonade), Türk Kahvesi (Turkish Coffee), Çay (Turkish Tea), Şerbet Çeşitleri (Traditional Fruit Sherbets)


### SequentialChain

In [46]:
prompt_template_name = PromptTemplate(
    input_variables=['cuisine'],
    template="I want to opan a resturant for {cuisine} food. Suggest a fancy name for this. Only one name"
)

name_chain = LLMChain(llm=model, prompt=prompt_template_name, output_key="resturant_name")

prompt_template_items = PromptTemplate(
    input_variables=['resturant_name'],
    template="Suggest some menu items for {resturant_name}. Return it as a comma separated list."
)
food_items_chain = LLMChain(llm=model, prompt=prompt_template_items, output_key="menu_items")

In [47]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['resturant_name', 'menu_items']
)
chain({'cuisine' : "Uzbek"})   

{'cuisine': 'Uzbek',
 'resturant_name': 'Azure Silk',
 'menu_items': 'Azure Tuna Tataki with Yuzu Pearls, Silk-Wrapped Shrimp Dumplings, Blue Lotus Spring Rolls, Cloud Whisper Edamame, Sapphire Noodle Salad with Grilled Chicken, Ocean Breeze Seabass with Forbidden Rice, Silken Tofu & Vegetable Stir-Fry, Pearl Noodle Bowl with Miso Glazed Salmon, Azure Berry Panna Cotta, Cloud Meringue with Lavender Cream, Silken Chocolate Lava Cake with Matcha Dust, Blue Moon Macaron Selection'}